In [1]:
import os
import numpy as np
import pandas as pd

In [2]:
Group1_scores = []
Group2_scores = []
Group3_scores = []

def emotion_definer(emotions):
    emotions = emotions.split(";")
    emotions = [x.strip(' ') for x in emotions]
    emotion_score = []
    for emotion in emotions:
        emotion = emotion.lower()
        if emotion == 'happy' or emotion == 'immersed' or emotion == 'excited' or emotion == 'hyped up' or emotion == 'hypedup' or emotion == 'delighted':
            emotion_score.append(2)
        elif emotion == 'sad' or emotion == 'angry' or ('frustrated' in emotion) or ('annoyed' in emotion) or emotion == 'bored' or emotion == 'sleepy' or emotion == 'tired' or emotion == 'nervous' or ('anxious' in emotion) or emotion == 'scared':
            emotion_score.append(0)
        elif emotion == 'tense' or emotion == 'confused' or  emotion == 'relaxed' or emotion == 'intrigued' or emotion == 'interest' or emotion == 'content' or emotion == 'calm' or emotion == 'mellow' or emotion == 'alright' or emotion == 'fine' or emotion == 'other':
            emotion_score.append(1)

    if not emotion_score:
        emotion_score.append(0)

    emotion = round(np.mean(emotion_score))

    if emotion is np.nan:
        emotion = 0

    return emotion




def load_Scores(location):
    print(location)
    folder = location[:-19]
    print(folder)
    var = os.listdir(folder)
    group = ''

    if "Group 1" in location:
        group = "Group 1"
    elif "Group 2" in location:
        group = "Group 2"
    elif "Group 3" in location:
        group = "Group 3"
        folder = folder + "/"

    for file in var:
        if group in file:
            if "xlsx" in file:
                location = folder + file
                data = pd.read_excel(location)
            elif "csv" in file:
                location = folder + file
                data = pd.read_csv(location)


    if "How would you rate this performance?" in data.columns:
        order = data["Watch ID"]
        scores = data["How would you rate this performance?"]
    elif "How did you rate the performance?" in data.columns:
        order = data["Watch ID"]
        scores  = data["How did you rate the performance?"]
    elif "How would you rate this performance?（10 is the best）" in data.columns:
        order = data["ID"]
        scores  = data["How would you rate this performance?（10 is the best）"]

    if "How would you describe your feelings through the performance?" in data.columns:
        emotions = data["How would you describe your feelings through the performance?"]
    elif "How do you describe your feeling during the programme? You may make multiple choices.\n" in data.columns:
        emotions = data["How do you describe your feeling during the programme? You may make multiple choices.\n"]

    emotions = emotions.apply(emotion_definer)

    return scores, order, emotions


def match_scores_to_data(scores, scores_order, emotions, data, data_order):
    matched_scores = []
    matched_data = []
    matched_emotions = []

    # Zip the scores and order and the data and data order
    scores_zip = zip(scores_order, scores, emotions)
    data_zip = zip(data_order, data)

    # Sort the zipped lists by order
    scores_zip = sorted(scores_zip, key=lambda x: x[0])
    data_zip = sorted(data_zip, key=lambda x: x[0])

    # Unzip the sorted lists
    sorted_scores_order, sorted_scores, sorted_emotions = zip(*scores_zip)
    sorted_data_order, sorted_data = zip(*data_zip)

    # Match the scores to the data using the order of each
    for i in range(min(len(sorted_scores_order), len(sorted_data_order))):
        if sorted_scores_order[i] == sorted_data_order[i]:
            matched_scores.append(sorted_scores[i])
            matched_data.append(sorted_data[i])
            matched_emotions.append(emotions[i])
    return matched_scores, matched_data, matched_emotions


def load_data(dir):
    data = []
    order = []
    for file in os.listdir(dir):
        if file.endswith(".csv"):
            print(os.path.join(dir, file))
            df = pd.read_csv(os.path.join(dir, file),on_bad_lines='skip')['DATA']
            if not df.empty:
                order.append(int(file[:-4]))
                data.append(df)

    shortest_dat = min([len(x) for x in data])
    for i in range(len(data)):
        data[i] = data[i].iloc[:shortest_dat]
        
        # Cut each item in data to the desired length
        sampling_rate = 24
        desired_length = 20*60*sampling_rate
        start_index = (len(data[i]) - desired_length) // 4
        end_index = start_index + desired_length
        data[i] = data[i].iloc[start_index:end_index]

    return data, order



try:
    Group1_scores, Group1_scores_order, Group1_emotions = load_Scores("/mnt/ArtHeart/Data/Group 1 Scores.xlsx")
    Group2_scores,Group2_scores_order, Group2_emotions = load_Scores("/mnt/ArtHeart/Data/Group 2 Scores.xlsx")
    Group3_scores,Group3_scores_order, Group3_emotions = load_Scores("/mnt/ArtHeart/Data/Group 3 Scores.csv")
    Group2, Group2_order = load_data("/mnt/ArtHeart/Data/Group 2")
    Group3, Group3_order = load_data("/mnt/ArtHeart/Data/Group 3")
except:
    try:
        Group1_scores, Group1_scores_order, Group1_emotions = load_Scores("W:\PhD\ArtHeart\Data\Group 1 Scores.xlsx")
        Group2_scores,Group2_scores_order, Group2_emotions = load_Scores("W:\PhD\ArtHeart\Data\Group 2 Scores.xlsx")
        Group3_scores,Group3_scores_order, Group3_emotions = load_Scores("W:\PhD\ArtHeart\Data\Group 3 Scores.csv")
        Group2_processed, Group3_processed, Group2_order, Group3_order = load_data("W:\PhD\ArtHeart\Data")

    except:
        Group1_scores, Group1_scores_order, Group1_emotions = load_Scores("Data/Group 1 Scores.xlsx")
        Group2_scores,Group2_scores_order, Group2_emotions = load_Scores("Data/Group 2 Scores.xlsx")
        Group3_scores,Group3_scores_order, Group3_emotions = load_Scores("Data/Group 3 Scores.csv")
        Group2_processed, Group3_processed, Group2_order, Group3_order = load_data("Data")



#Group1_scores, Group1_processed, Group1_emotions = match_scores_to_data(Group1_scores, Group1_scores_order, Group1_emotions, Group1_processed, Group1_order)
Group2_scores, Group2_processed, Group2_emotions = match_scores_to_data(Group2_scores, Group2_scores_order, Group2_emotions, Group2, Group2_order)
Group3_scores, Group3_processed, Group3_emotions = match_scores_to_data(Group3_scores, Group3_scores_order, Group3_emotions, Group3, Group3_order)


print(len(Group2_processed))
print(Group2_emotions)

/mnt/ArtHeart/Data/Group 1 Scores.xlsx
/mnt/ArtHeart/Data/
W:\PhD\ArtHeart\Data\Group 1 Scores.xlsx
W:\PhD\ArtHeart\Data\
Data/Group 1 Scores.xlsx
Data/


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [3]:
from sklearn.preprocessing import StandardScaler

def scale_data(data):
    scaler = StandardScaler()
    for i in range(len(data)):
        data[i]= np.array(data[i]).reshape(-1,1)
        scaled_data = scaler.fit_transform(data[i])
        data[i] = scaled_data
    return data

#Group1_processed = scale_data(Group1_processed)
Group2_processed = scale_data(Group2_processed)
Group3_processed = scale_data(Group3_processed)

In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

data_all = []
data_all.extend(Group2_processed)
data_all.extend(Group3_processed)
data_all = np.array(data_all)

scores_all = []
scores_all.extend(Group2_emotions)
scores_all.extend(Group3_emotions)

#data_all = np.expand_dims(data_all, axis=-1)

# Convert scores to one-hot encoding
scores_all = tf.keras.utils.to_categorical(scores_all, num_classes=3)

X_train, X_test, y_train, y_test = train_test_split(data_all, scores_all, test_size=0.2, random_state=66)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, input_shape=X_train[0].shape),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=100, validation_split=0.1, batch_size=4)



2023-06-28 15:02:13.787628: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                4352      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 4,931
Trainable params: 4,931
Non-trainable params: 0
_________________________________________________________________
Train on 43 samples, validate on 5 samples


2023-06-28 15:02:14.697996: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcuda.so.1
2023-06-28 15:02:14.702395: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1082] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-28 15:02:14.702517: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1674] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:04:00.0
2023-06-28 15:02:14.702531: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcudart.so.12
2023-06-28 15:02:14.714346: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcublas.so.12
2023-06-28 15:02:14.715724: I tensorflow/stream_executor/platform/default/dso_loader.cc:50] Successfully opened dynamic library libcuff

In [25]:
model.evaluate(X_test, y_test)


12/12 [==============================] - 0s 107us/sample - loss: 3.9396 - acc: 0.3333


[3.9396307468414307, 0.33333334]